<a href="https://colab.research.google.com/github/Manaswini1912/INFO-5731/blob/main/Kodela_Manaswini_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [25]:
pip install IMDbPY

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 5.3 MB/s eta 0:00:00


In [20]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def retrieve_reviews(movie_id):
    base_url = f'https://www.imdb.com/title/tt15398776/reviews'
    reviews_list = []
    try:
        while True:
            response = requests.get(base_url)
            soup = BeautifulSoup(response.content, 'html.parser')

            containers = soup.find_all('div', class_='review-container')

            for container in containers:
                title_element = container.find('a', class_='title')
                title = title_element.get_text(strip=True) if title_element else 'No Title'

                review_text_element = container.find('div', class_='text show-more__control')
                review_text = review_text_element.get_text(strip=True) if review_text_element else 'No Review Text'

                reviews_list.append((title, review_text))

            has_more_reviews = len(reviews_list) < 1000 and containers
            if not has_more_reviews:
                break

            time.sleep(1)

    except Exception as e:
        print(f'Error occurred: {e}')

    return reviews_list[:1000]

movie_ids = ['tt15398776']
all_reviews = []

for movie_id in movie_ids:
    all_reviews.extend(retrieve_reviews(movie_id))
    if len(all_reviews) >= 1000:
        break

with open('movie_review_feedback.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Review'])
    for title, review in all_reviews:
        writer.writerow([title, review])

print(f'Successfully saved {len(all_reviews)} reviews with titles.')


Successfully saved 1000 reviews with titles.


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [21]:
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    cleaned_text = ''.join(char for char in text if char.isalnum() or char.isspace())
    cleaned_text = ''.join(char for char in cleaned_text if not char.isdigit())
    cleaned_text = cleaned_text.lower()
    return cleaned_text

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_text = ' '.join(word for word in words if word.lower() not in stop_words)
    return filtered_text

def stem_text(text):
    ps = PorterStemmer()
    stemmed_text = ' '.join(ps.stem(word) for word in text.split())
    return stemmed_text

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    return lemmatized_text

def process_reviews(reviews_data):
    processed_reviews = []
    for title, review in reviews_data:
        processed_title = clean_text(title)
        processed_review = clean_text(review)
        processed_reviews.append((processed_title, processed_review))
    return processed_reviews

# Input and output file paths
input_file_path = 'movie_review_feedback.csv'
output_file_path = 'cleaned_reviews.csv'

# Read existing reviews from the input CSV file
reviews_data = []

with open(input_file_path, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)  # Skip header
    for row in reader:
        title, review = row
        reviews_data.append((title, review))

# Clean and process the reviews
processed_reviews = process_reviews(reviews_data)

# Save cleaned reviews to a new CSV file
with open(output_file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Cleaned Title', 'Cleaned Review'])  # Header
    for processed_title, processed_review in processed_reviews:
        writer.writerow([processed_title, processed_review])

# Print the number of cleaned reviews saved
print(f'Successfully saved {len(processed_reviews)} cleaned reviews in {output_file_path}.')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Successfully saved 1000 cleaned reviews in cleaned_reviews.csv.


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [24]:
import spacy

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Read the cleaned reviews from the CSV file
cleaned_reviews = []

with open('cleaned_reviews.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)  # Skip header
    for row in reader:
        cleaned_title, cleaned_review = row
        cleaned_reviews.append((cleaned_title, cleaned_review))

# Choose a sample review for analysis
sample_review = cleaned_reviews[0][1]  # Choose the review text from the first row

# (1) Parts of Speech (POS) Tagging
doc = nlp(sample_review)
pos_tags_count = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}

for token in doc:
    if token.pos_ == 'NOUN':
        pos_tags_count['Noun'] += 1
    elif token.pos_ == 'VERB':
        pos_tags_count['Verb'] += 1
    elif token.pos_ == 'ADJ':
        pos_tags_count['Adjective'] += 1
    elif token.pos_ == 'ADV':
        pos_tags_count['Adverb'] += 1

print('POS Tags Count:', pos_tags_count)

# (2) Constituency Parsing and Dependency Parsing
# Print constituency parsing tree
print('\nConstituency Parsing Tree:')
for sent in doc.sents:
    for token in sent:
        print(f"{token.text} <--{token.dep_}-- {token.head.text}")

# (3) Named Entity Recognition (NER)
entities_count = {'PERSON': 0, 'ORG': 0, 'LOC': 0, 'PRODUCT': 0, 'DATE': 0}

for ent in doc.ents:
    if ent.label_ in entities_count:
        entities_count[ent.label_] += 1
    else:
        entities_count[ent.label_] = 1  # If the key doesn't exist, create it and set count to 1


print('\nNamed Entity Recognition (NER) Count:')
for entity, count in entities_count.items():
    print(f"{entity}: {count}")

POS Tags Count: {'Noun': 60, 'Verb': 50, 'Adjective': 38, 'Adverb': 33}

Constituency Parsing Tree:
one <--nsubj-- included
of <--prep-- one
the <--det-- films
most <--advmod-- anticipated
anticipated <--amod-- films
films <--pobj-- of
of <--prep-- films
the <--det-- year
year <--pobj-- of
for <--prep-- films
many <--amod-- people
people <--pobj-- for
myself <--appos-- people
included <--ROOT-- included
oppenheimer <--nsubj-- delivers
largely <--advmod-- delivers
delivers <--ccomp-- included
much <--dobj-- delivers
of <--prep-- much
its <--poss-- great
great <--pobj-- of
i <--nsubj-- feel
feel <--ccomp-- delivers
like <--mark-- loved
i <--nsubj-- loved
loved <--advcl-- feel
two <--dobj-- loved
of <--prep-- two
its <--poss-- hours
three <--nummod-- hours
hours <--pobj-- of
and <--cc-- loved
liked <--conj-- loved
the <--det-- hour
other <--amod-- hour
hour <--dobj-- liked
but <--cc-- delivers
its <--poss-- fact
that <--det-- fact
fact <--conj-- delivers
that <--nsubj-- stops
stops <--acl

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# I tried to complete at my best with all my efforts, but I found questions quite challenging with the included concepts. But I found Retreiving the reviews of a movie is quite interesting and fun to do.